# Derive Rotation Matrix

In [ ]:
import numpy as np
import sympy as sp
from sympy.printing import fcode, numpy

%matplotlib widget

We define the angles $\theta $ (inclination) and $\phi $ (twist) and the rotation matrices.

In [ ]:
theta, phi = sp.symbols('theta, phi', real=True)
x, y, z = sp.symbols('x, y, z', real=True)

def Rx(angle):
    return sp.Matrix([
    [1,       0,              0],
    [0,       sp.cos(angle), -sp.sin(angle)],
    [0,       sp.sin(angle),  sp.cos(angle)],
])

def Ry(angle):
    return sp.Matrix([
    [ sp.cos(angle), 0, sp.sin(angle)],
    [0,              1, 0],
    [-sp.sin(angle), 0, sp.cos(angle)],
])

def Rz(angle):
    return sp.Matrix([
    [sp.cos(angle), -sp.sin(angle), 0],
    [sp.sin(angle),  sp.cos(angle), 0],
    [0,              0,             1],
])

Define the point $p_0=\begin{pmatrix}x\\y\\z\end{pmatrix}$ and then the rotation matrix and it's inverse. $p_1$ and $p_2$ are then the rotated (or un-rotated) coordinates of a point.

In [ ]:
p0 = sp.Matrix([x, y, z])

M = Rz(phi) * Rx(theta)

p1 = sp.simplify(M) * p0
p2 = sp.simplify(M.inv()) * p0

Print it in a way that we can easily turn into code

In [ ]:
printer=numpy.NumPyPrinter()
print('Rotation:')
for v in p1: print(printer.doprint(v).replace('numpy', 'np'))

print('\nDe-Rotation:')
for v in p2: print(printer.doprint(v).replace('numpy', 'np'))

# Transformation

In [ ]:
def warp(x, y, z, phi, theta):
    xprime = x * np.cos(phi)   - y * np.sin(phi) * np.cos(theta) + z * np.sin(phi)   * np.sin(theta)
    yprime = x * np.sin(phi)   + y * np.cos(phi) * np.cos(theta) - z * np.sin(theta) * np.cos(phi)
    zprime = y * np.sin(theta) + z * np.cos(theta)
    return xprime, yprime, zprime

In [ ]:
def unwarp(x, y, z, phi, theta):
    xprime =  x * np.cos(phi) + y * np.sin(phi)
    yprime = -x * np.sin(phi) * np.cos(theta) + y * np.cos(phi)   * np.cos(theta) + z * np.sin(theta)
    zprime =  x * np.sin(phi) * np.sin(theta) - y * np.sin(theta) * np.cos(phi)   + z * np.cos(theta)
    return xprime, yprime, zprime

In [ ]:
def logistic(a, r, r0, dr):
    return np.radians( a / (1 + np.exp((r - r0) / (0.1 * dr))))

In [ ]:
incw = 60.
PAw = 60.

In [ ]:
nphi = 60
nr = 15
r0 = np.linspace(0.8, 1.2, nr)
phi0 = np.linspace(0, 2 * np.pi, nphi)
R, PHI = np.meshgrid(r0, phi0, indexing='ij')

In [ ]:
x = R * np.cos(PHI)
y = R * np.sin(PHI)
z = np.zeros_like(y)

In [ ]:
inc_a     = logistic(incw, r0, 1, 0.4) ### Specify the r0 and dr in AU 
PA_a      = logistic(PAw, r0, 1, 0.4)

In [ ]:
xw, yw, zw = warp(x, y, z, inc_a[:, None], PA_a[:, None])

In [ ]:
xu, yu, zu = unwarp(xw, yw, zw, inc_a[:, None], PA_a[:, None])

In [ ]:
f = plt.figure()
ax = f.add_subplot(projection='3d')
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.set_zlim(-1, 1)

for ir in range(nr):
    ax.plot(x[ir, :], y[ir, :], z[ir,:], 'k')
    ax.plot(xw[ir, :], yw[ir, :], zw[ir,:], 'r-', lw=1)
    ax.plot(xu[ir, :], yu[ir, :], zu[ir,:], 'g--', lw=1)